In [89]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb



#評価用

train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

with open('ensamble/rf_train.pickle', mode='br') as fi:
    train_p1 = pickle.load(fi)
    
with open('ensamble/lgb_train.pickle', mode='br') as fi:
    train_p2 = pickle.load(fi)





#データ読み込み
with open('ensamble/rf_test.pickle', mode='br') as fi:
    test_p1 = pickle.load(fi)
    
with open('ensamble/lgb_test.pickle', mode='br') as fi:
    test_p2 = pickle.load(fi)

In [90]:
df = pd.concat([train['Outcome'],train_p1['pred'],train_p2['pred']], axis=1, ignore_index=True).rename(columns={0: 'true', 1: 'pred1', 2: 'pred2'})
df['ensemble'] = df['pred1']*0.5 + df['pred2']*0.5


def evaluate_ensemble(input_df, col_pred):
    print('[auc] model1:{:.4f}, model2:{:.4f}, -> emsamble]{:.4f}'.format(
        roc_auc_score(input_df['true'], input_df['pred1']),
        roc_auc_score(input_df['true'], input_df['pred2']),
        roc_auc_score(input_df['true'], input_df[col_pred]),
    ))

In [91]:
evaluate_ensemble(df, col_pred='ensemble')

[auc] model1:0.8053, model2:0.8182, -> emsamble]0.8192


In [92]:
df = pd.concat([test['index'],test_p1['pred'],test_p2['pred']], axis=1, ignore_index=True).rename(columns={0: 'index', 1: 'pred1', 2: 'pred2'})
df['ensemble'] = df['pred1']*0.5 + df['pred2']*0.5
df

,index,pred1,pred2,ensemble
0,398,0.339819,0.217108,0.278463
1,3833,0.079638,0.074849,0.077244
2,4836,0.036652,0.079949,0.058300
3,4572,0.418552,0.373731,0.396141
4,636,0.211312,0.137988,0.174650
...,...,...,...,...
1995,3138,0.212670,0.123821,0.168245
1996,191,0.143439,0.186012,0.164726
1997,3294,0.634389,0.567065,0.600727
1998,3073,0.242534,0.190744,0.216639


In [93]:
sub = df.copy()    
   
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 

sub.to_csv('sub/submission_emsamble.csv', index=None, header=False,)